In [121]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from PIL import Image
from IPython.display import Image, display
from sklearn.datasets import load_iris

In [122]:
def x_round(x, numpixels = 1):
    # round value to nearest pixel (one pixel is 1/50 A)
    return round(x*50/numpixels)/(50/numpixels)

In [123]:
directory = '/mnt_home/zoek/code/APF-BL-DAP/Zoe/6sResults/'
spectra_file = directory + 's_detectionWLs.csv'
residuals_file = directory + 'r_detectionWLs.csv'

spectra_results = pd.read_csv(spectra_file)
residuals_results = pd.read_csv(residuals_file)

persistent_spectra = pd.read_csv(directory + 's_persistent_events.csv')
persistent_residuals = pd.read_csv(directory + 'r_persistent_events.csv')

In [124]:
# repeated wavelengths using the 3 AMD run - in the rest frame and observed frame

rep_observed = pd.read_csv('/mnt_home/zoek/code/APF-BL-DAP/Zoe/3sResults/rep_observed_wls.csv')
rep_rest = pd.read_csv('/mnt_home/zoek/code/APF-BL-DAP/Zoe/3sResults/rep_rest_wls.csv')

rep_observed_wls_3s = rep_observed['observed wl']
rep_rest_wls_3s = rep_rest['rest wl']

rep_observed_wls_3s = x_round(rep_observed_wls_3s)
rep_rest_wls_3s = x_round(rep_rest_wls_3s)

In [126]:
# These are the 55 events that passed all the tests.
persistent_residuals.to_csv('pers_res.csv')

## Repeated observed wavelengths - sky emission lines, deblazing artifacts

In [127]:
# Remove all candidates with observed wavelengths that fall into rep_observed_wls_3s
persistent_residuals_rem_obs = persistent_residuals[~persistent_residuals['rounded r unshifted wl'].isin(rep_observed_wls_3s)]
persistent_residuals_rem_obs.reset_index(inplace=True)

In [128]:
print('These are the 9 TOTAL observed wavelengths that are removed:')
removed_obs_wls = persistent_residuals[persistent_residuals['rounded r unshifted wl'].isin(rep_observed_wls_3s)]
removed_obs_wls = removed_obs_wls['rounded r unshifted wl'].unique()
removed_obs_wls

These are the 9 TOTAL observed wavelengths that are removed:


array([5013.88, 5014.32, 5016.48, 5083.5 , 5405.88, 5577.4 , 5577.42,
       5895.96, 5895.98])

In [129]:
print('deblazing artifacts:')
print([5013.88, 5014.32, 5016.48])
print('')
print('sky emission lines to classify:')
print([5083.5 , 5405.88, 5577.4 , 5577.42, 5895.96, 5895.98])

deblazing artifacts:
[5013.88, 5014.32, 5016.48]

sky emission lines to classify:
[5083.5, 5405.88, 5577.4, 5577.42, 5895.96, 5895.98]


In [130]:
# drop all candidates that have less than 3 observations in a night
less_than_3 = persistent_residuals_rem_obs.groupby(['rounded r shifted wl', 'star']).count().reset_index()
less_than_3 = less_than_3[less_than_3['identifier'] < 3]
less_than_3.reset_index(inplace=True)
less_than_3 = less_than_3.loc[:, ['rounded r shifted wl', 'star']]
for index, row in less_than_3.iterrows():
    wl = row['rounded r shifted wl']
    star = row['star']
    sub_table = persistent_residuals_rem_obs[persistent_residuals_rem_obs['star'] == star]
    sub_table = sub_table[sub_table['rounded r shifted wl'] == wl]
    indicies_to_remove = list(sub_table['index'])
    persistent_residuals_rem_obs = persistent_residuals_rem_obs[~persistent_residuals_rem_obs['index'].isin(indicies_to_remove)]

In [132]:
# These are all the events after removing the observed wavelengths that show up repeatedly across the 3s results.
persistent_residuals_rem_obs.to_csv('pers_res_filtered.csv')

In [133]:
print('# final events')
len(persistent_residuals_rem_obs.groupby(['star', 'rounded r shifted wl']).first().reset_index())

# final events


42

In [134]:
print('# stars')
len(persistent_residuals_rem_obs['star'].unique())

# stars


16

In [151]:
persistent_residuals['date'] = persistent_residuals['observation time'].str.split('T').str[0]
dates = persistent_residuals.groupby(['star', 'date']).first()[['identifier']].reset_index()
dates = dates.groupby('star').agg(lambda x: list(x)).reset_index().drop(columns='identifier')

In [152]:
stars = list(dates['star'])
dates = list(dates['date'])

In [154]:
stars

['HIP101262',
 'HIP107975',
 'HIP114430',
 'HIP117463',
 'HIP14286',
 'HIP14576',
 'HIP24332',
 'HIP36152',
 'HIP45836',
 'HIP47990',
 'HIP50384',
 'HIP67301',
 'HIP677',
 'HIP68030',
 'HIP73695',
 'HIP77760',
 'HIP84862',
 'HIP8543',
 'HIP91919',
 'HIP91926',
 'HIP93747',
 'HIP97649',
 'HIP98677',
 'TIC142276270',
 'TIC198456933',
 'TIC219778329']

In [155]:
dates

[['2020-05-28'],
 ['2019-07-10'],
 ['2018-04-02'],
 ['2019-11-06'],
 ['2018-11-05'],
 ['2018-11-05'],
 ['2016-11-02'],
 ['2018-03-28'],
 ['2018-06-19'],
 ['2021-04-11'],
 ['2017-03-08'],
 ['2018-07-27'],
 ['2018-10-23'],
 ['2017-05-14', '2020-05-29', '2020-06-05'],
 ['2018-05-22'],
 ['2018-06-06'],
 ['2019-07-10', '2019-07-24'],
 ['2020-08-15'],
 ['2017-08-26'],
 ['2017-08-25'],
 ['2018-06-19'],
 ['2018-06-29'],
 ['2020-05-23'],
 ['2021-06-20'],
 ['2021-08-31'],
 ['2021-08-27']]